In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import similarities
from gensim import corpora, models, similarities

# spacy for lemmatization
import spacy

import nltk
from nltk.corpus import stopwords 
import string
from nltk import word_tokenize

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
# f = open("/Users/dhams/Desktop/Research/PreProcessing/plot-genie-index-stop-removal.txt","r")
# g = open("/Users/dhams/Desktop/Research/PreProcessing/clean-romance-pg.txt","r")
# h = open("/Users/dhams/Desktop/Research/PreProcessing/adventure-pg-clean.txt","r")
# i = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-argosy-1917.txt","r")
# j = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Dime-Detective.txt","r")
# k = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Popular-Detective.txt","r")
# l = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Science_Wonder_Stories.txt","r")
# m = open("/Users/dhams/Desktop/algo-detection/pre-processing/clean-Terror_Tales.txt","r")
f = open("../pre-processing/combined/combined-everything.txt","r")

In [4]:
# lines_1 = f.readlines()
# lines_2 = g.readlines()
# lines_3 = h.readlines()
# lines_4 = i.readlines()
# lines_5 = j.readlines()
# lines_6 = k.readlines()
# lines_7 = l.readlines()
# lines_8 = m.readlines()
# f.close()
# g.close()
# h.close()
# i.close()
# j.close()
# k.close()
# l.close()
# m.close()
# lines = lines_4 + lines_5 + lines_7 + lines_8 + lines_1 + lines_2 + lines_3 + lines_6
lines = f.readlines()

In [5]:
def send_tokens(line):
    tokens = word_tokenize(line)
    return tokens

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatization(texts):
    lemmatizer = WordNetLemmatizer()
    lemmatized_word_list = []
    for line in texts:
        tokens = send_tokens(line)
        lemmatized_word_list.append([lemmatizer.lemmatize(w) for w in tokens])        
    return lemmatized_word_list

In [7]:
lemmatized_tokens = lemmatization(lines)

In [8]:
id2word = corpora.Dictionary(lemmatized_tokens)

# Create Corpus
texts = lemmatized_tokens

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [10]:
topics = lda_model.print_topics()
doc_lda = lda_model[corpus]

In [11]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_tokens, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.7336256168954

Coherence Score:  0.3755014699399275


In [12]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.371699  0.256885       1        1  20.709702
18    -0.317356 -0.166461       2        1  14.423940
12    -0.118296 -0.178990       3        1   6.841399
6     -0.054175 -0.058754       4        1   6.240233
14     0.003365 -0.005199       5        1   5.178404
19     0.002147 -0.028860       6        1   4.696590
1      0.037254 -0.009837       7        1   3.915734
10     0.031283 -0.009431       8        1   3.912164
15     0.016837 -0.030994       9        1   3.839548
8      0.036021 -0.008649      10        1   3.745821
4      0.037080 -0.012236      11        1   3.638186
16     0.044227  0.014095      12        1   3.572661
17     0.073925  0.028492      13        1   3.120918
13     0.065516  0.017826      14        1   2.756562
0      0.078259  0.036063      15        1   2.625350
3      0.085403  0.031882      16        1   2.472166
11     0.078720  0.030451      17        1   2.388459
9      0.090111  0.031405      18        1   2.212945
5      0.088500  0.029570      19        1   1.998399
2      0.092880  0.032741      20        1   1.710821, topic_info=      Category         Freq         Term        Total  loglift  logprob
term                                                                   
11     Default  2723.000000        story  2723.000000  30.0000  30.0000
619    Default  2830.000000          man  2830.000000  29.0000  29.0000
1838   Default  2352.000000        would  2352.000000  28.0000  28.0000
629    Default  4148.000000          one  4148.000000  27.0000  27.0000
316    Default  1969.000000         back  1969.000000  26.0000  26.0000
893    Default  1887.000000         time  1887.000000  25.0000  25.0000
2589   Default  1989.000000          day  1989.000000  24.0000  24.0000
541    Default  1615.000000         hand  1615.000000  23.0000  23.0000
2024   Default  1506.000000          new  1506.000000  22.0000  22.0000
591    Default  1843.000000        money  1843.000000  21.0000  21.0000
338    Default  2075.000000          old  2075.000000  20.0000  20.0000
573    Default  1658.000000          eye  1658.000000  19.0000  19.0000
3524   Default  1881.000000         said  1881.000000  18.0000  18.0000
475    Default  1418.000000         mark  1418.000000  17.0000  17.0000
845    Default  1360.000000         upon  1360.000000  16.0000  16.0000
2918   Default  1487.000000         free  1487.000000  15.0000  15.0000
2710   Default  1625.000000         like  1625.000000  14.0000  14.0000
697    Default  1150.000000         come  1150.000000  13.0000  13.0000
4019   Default  2227.000000         send  2227.000000  12.0000  12.0000
4549   Default  2712.000000         tale  2712.000000  11.0000  11.0000
2118   Default  1367.000000        could  1367.000000  10.0000  10.0000
477    Default  1287.000000          two  1287.000000   9.0000   9.0000
2601   Default  1109.000000         last  1109.000000   8.0000   8.0000
1342   Default  1106.000000         long  1106.000000   7.0000   7.0000
3497   Default  1190.000000          men  1190.000000   6.0000   6.0000
2783   Default  1096.000000          pay  1096.000000   5.0000   5.0000
935    Default  1156.000000         face  1156.000000   4.0000   4.0000
3229   Default  1059.000000        shall  1059.000000   3.0000   3.0000
1924   Default  1331.000000         life  1331.000000   2.0000   2.0000
3318   Default  1064.000000          see  1064.000000   1.0000   1.0000
...        ...          ...          ...          ...      ...      ...
73419  Topic20     3.113628    abyssinia    76.601807   0.8654  -8.2053
6367   Topic20     2.946865   represents    72.415207   0.8665  -8.2603
73462  Topic20     3.372572        fungi    83.459381   0.8595  -8.1254
47127  Topic20     5.971154    hollywood   151.560654   0.8342  -7.5541
73675  Topic20     5.441066       dunham   138.465973   0.8315  -7.6471
69615  Topic20     2.915016   ent

In [14]:
index = similarities.MatrixSimilarity(doc_lda.corpus)

In [15]:
s = open("../pre-processing/combined/combined-source.txt","r")
source_lines = s.readlines()
source_bow = [id2word.doc2bow(text.lower().split()) for text in source_lines]
source_lda = lda_model[source_bow]

In [16]:
index = similarities.MatrixSimilarity(source_lda.corpus)

In [18]:
sus_test = open("../pre-processing/combined/combined-test-suspected-algo.txt","r")
sus_test_lines = sus_test.readlines()
sus_test_bow = [id2word.doc2bow(text.lower().split()) for text in sus_test_lines]
sus_test_lda = lda_model[sus_test_bow]

In [ ]:
index.save("simIndex.index")

In [19]:
sims = index[sus_test_lda.corpus]
#sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
for i in sims[200]:
    if i>0:
        print(i)

In [21]:
for j in range(0,len(sus_test_lines)):
    #print("Similarity of sentence:",test_lines[j])
    for i in range(0,len(sims[j])):
        if sims[j][i]>0.5:
            print((i+1),".",lines[i],":",sims[j][i])

1779 . story 
 : 0.6666667
2697 . story 
 : 0.6666667
1068 . guest 
 : 0.57735026
5274 . guest 
 : 0.57735026
39 . professor 
 : 0.57735026
118 . professor 
 : 0.57735026
2732 . professor 
 : 0.57735026
2809 . professor 
 : 0.57735026
4408 . professor 
 : 0.57735026
4636 . professor 
 : 0.57735026
5435 . professor 
 : 0.57735026
10125 . professor 
 : 0.57735026
12425 . professor 
 : 0.57735026
841 . plan 
 : 0.57735026
14110 . steal plan 
 : 0.81649655
14637 . building 
 : 0.57735026
14760 . building 
 : 0.57735026
14768 . building 
 : 0.57735026
14780 . building 
 : 0.57735026
15277 . building 
 : 0.57735026
15316 . building 
 : 0.57735026
15369 . building 
 : 0.57735026
15376 . building 
 : 0.57735026
15399 . building 
 : 0.57735026
15423 . building 
 : 0.57735026
15653 . building 
 : 0.57735026
15663 . building 
 : 0.57735026
15690 . building 
 : 0.57735026
15705 . building 
 : 0.57735026
15910 . building 
 : 0.57735026
15982 . building 
 : 0.57735026
16186 . building 
 : 0.57735026

8506 . come quick
 : 0.70710677
8700 . ohmr randolph come
 : 0.57735026
8778 . come dont mean
 : 0.57735026
8823 . little fool
 : 0.70710677
8916 . knew dan fool ididnt think much fool
 : 0.6666667
2429 . thought 
 : 0.70710677
7450 . desire punish scoundrel
 : 0.57735026
3237 . need obtain judgment 
 : 0.5163978
3365 . avoid judgment 
 : 0.6324555
14253 . judgment 
 : 0.8944272
14344 . judgment 
 : 0.8944272
14410 . erroneous judgment 
 : 0.6324555
8378 . said quite
 : 0.70710677
8497 . good reasontoo said
 : 0.57735026
8578 . said
 : 1.0
8693 . ill julie said
 : 0.57735026
8697 . hello said
 : 0.70710677
8726 . fall ellen said
 : 0.57735026
8731 . said hedidnt give hoot mother said
 : 0.70710677
8873 . said
 : 1.0
9036 . evelyn said
 : 0.70710677
9037 . said emphatically unequivocally
 : 0.57735026
9043 . already said toomuch
 : 0.57735026
9146 . said
 : 1.0
9174 . said dream
 : 0.70710677
7016 . dream
 : 1.0
9107 . another dream
 : 0.70710677
9174 . said dream
 : 0.70710677
4928 . b

3446 . world would
 : 0.70710677
8794 . world aint badkid
 : 0.57735026
9663 . world war 
 : 0.70710677
15094 . frozen world 
 : 0.70710677
15859 . frozen world 
 : 0.70710677
39 . professor 
 : 0.6324555
118 . professor 
 : 0.6324555
2732 . professor 
 : 0.6324555
2809 . professor 
 : 0.6324555
4408 . professor 
 : 0.6324555
4636 . professor 
 : 0.6324555
5435 . professor 
 : 0.6324555
10125 . professor 
 : 0.6324555
12425 . professor 
 : 0.6324555
1610 . giving wrong address 
 : 0.57735026
7902 . wrong address given
 : 0.57735026
8701 . there something wrong
 : 0.57735026
8990 . bill wrong
 : 0.70710677
9018 . bill wrong
 : 0.70710677
3700 . enough
 : 1.0
4353 . enough
 : 1.0
8578 . said
 : 0.57735026
8873 . said
 : 0.57735026
9146 . said
 : 0.57735026
8160 . one compelled sacrifice loved one another
 : 0.53033006
15286 . power 
 : 0.70710677
5704 . careful polite maudlin 
 : 0.57735026
5857 . dictatorial dogmatic careful 
 : 0.57735026
9007 . bill careful hair
 : 0.57735026
726 . pl

8768 . whats matter girlie
 : 0.6666666
2180 . person asked forgiveness 
 : 0.57735026
8501 . asked indifferently
 : 0.70710677
8624 . julie asked dryly
 : 0.57735026
8375 . well time know
 : 0.57735026
8445 . philosophy well put
 : 0.57735026
8473 . heknew jack well
 : 0.57735026
8633 . say well plutocrat
 : 0.57735026
8834 . well say
 : 0.70710677
15128 . burning oil well 
 : 0.57735026
15310 . burning oil well 
 : 0.57735026
15495 . oil well 
 : 0.70710677
15995 . burning oil well 
 : 0.57735026
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 : 0.70710677
4636 . professor 
 : 0.70710677
5435 . professor 
 : 0.70710677
10125 . professor 
 : 0.70710677
12425 . professor 
 : 0.70710677
8530 . exclaimed
 : 1.0
8589 . exclaimed recoiling
 : 0.70710677
39 . professor 
 : 0.57735026
118 . professor 
 : 0.57735026
2732 . professor 
 : 0.57735026
2809 . professor 
 : 0.57735026
4408 . professor 
 

7251 . thief
 : 1.0
7522 . combat ignorant thief
 : 0.57735026
14858 . cattle thief 
 : 0.70710677
14877 . thief 
 : 1.0
14974 . thief 
 : 1.0
15252 . execute thief 
 : 0.70710677
16168 . cattle thief 
 : 0.70710677
2410 . circumstance 
 : 0.57735026
13950 . circumstance 
 : 0.57735026
3489 . automobile way marry
 : 0.5163978
5557 . stand way union 
 : 0.5163978
6951 . way planned frighten
 : 0.5163978
6956 . way appeal made
 : 0.5163978
7003 . way soul saved
 : 0.5163978
7010 . way effort made
 : 0.5163978
9079 . couldnt squirm way
 : 0.5163978
11568 . stand way union 
 : 0.5163978
12668 . blaze way civilization 
 : 0.5163978
12743 . indebtedness stand way 
 : 0.5163978
12856 . religion stand way 
 : 0.5163978
12860 . necessity scouting way 
 : 0.5163978
4893 . guard royal court 
 : 0.5163978
4982 . guard royal court 
 : 0.5163978
12526 . guard royal court 
 : 0.5163978
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.

8511 . good
 : 0.70710677
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 : 0.70710677
4636 . professor 
 : 0.70710677
5435 . professor 
 : 0.70710677
10125 . professor 
 : 0.70710677
12425 . professor 
 : 0.70710677
12851 . aroused 
 : 0.70710677
184 . aboard ship 
 : 0.70710677
196 . aboard submarine 
 : 0.70710677
307 . aboard submarine 
 : 0.70710677
7058 . aboard ship
 : 0.70710677
7197 . aboard train
 : 0.70710677
13069 . invasion aboard ship 
 : 0.57735026
13266 . attack aboard ship 
 : 0.57735026
13320 . riot aboard ship 
 : 0.57735026
13381 . fire aboard ship 
 : 0.57735026
13441 . search aboard ship 
 : 0.57735026
13637 . fight aboard ship 
 : 0.57735026
15246 . aboard ship 
 : 0.70710677
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 : 0.70710677
4636 . professor 
 : 0.70710677
5435 . 

1818 . know 
 : 0.57735026
1944 . information 
 : 0.57735026
13970 . information 
 : 0.57735026
8506 . come quick
 : 0.70710677
8700 . ohmr randolph come
 : 0.57735026
8778 . come dont mean
 : 0.57735026
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 : 0.70710677
4636 . professor 
 : 0.70710677
5435 . professor 
 : 0.70710677
10125 . professor 
 : 0.70710677
12425 . professor 
 : 0.70710677
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 : 0.70710677
4636 . professor 
 : 0.70710677
5435 . professor 
 : 0.70710677
8578 . said
 : 0.70710677
8873 . said
 : 0.70710677
9146 . said
 : 0.70710677
10125 . professor 
 : 0.70710677
12425 . professor 
 : 0.70710677
39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 

39 . professor 
 : 0.70710677
118 . professor 
 : 0.70710677
2732 . professor 
 : 0.70710677
2809 . professor 
 : 0.70710677
4408 . professor 
 : 0.70710677
4636 . professor 
 : 0.70710677
5435 . professor 
 : 0.70710677
8530 . exclaimed
 : 0.70710677
10125 . professor 
 : 0.70710677
12425 . professor 
 : 0.70710677
2419 . justifiable 
 : 0.57735026
1571 . another 
 : 0.57735026
1789 . another 
 : 0.57735026
2179 . another 
 : 0.57735026
2532 . another 
 : 0.57735026
2429 . thought 
 : 0.70710677
2180 . person asked forgiveness 
 : 0.57735026
8501 . asked indifferently
 : 0.70710677
8624 . julie asked dryly
 : 0.57735026
1454 . sudden change indicated 
 : 0.6666666
39 . professor 
 : 0.57735026
118 . professor 
 : 0.57735026
2732 . professor 
 : 0.57735026
2809 . professor 
 : 0.57735026
4408 . professor 
 : 0.57735026
4636 . professor 
 : 0.57735026
5435 . professor 
 : 0.57735026
10125 . professor 
 : 0.57735026
12425 . professor 
 : 0.57735026
8592 . indeed demurred
 : 0.70710677
26

8375 . well time know
 : 0.57735026
8445 . philosophy well put
 : 0.57735026
8473 . heknew jack well
 : 0.57735026
8633 . say well plutocrat
 : 0.57735026
8834 . well say
 : 0.70710677
15128 . burning oil well 
 : 0.57735026
15310 . burning oil well 
 : 0.57735026
15495 . oil well 
 : 0.70710677
15995 . burning oil well 
 : 0.57735026
6805 . plain 
 : 0.70710677
7143 . plain
 : 0.70710677
9530 . plain 
 : 0.70710677
39 . professor 
 : 0.57735026
118 . professor 
 : 0.57735026
2732 . professor 
 : 0.57735026
2809 . professor 
 : 0.57735026
4408 . professor 
 : 0.57735026
4636 . professor 
 : 0.57735026
5435 . professor 
 : 0.57735026
10125 . professor 
 : 0.57735026
12425 . professor 
 : 0.57735026
2429 . thought 
 : 1.0
39 . professor 
 : 0.57735026
118 . professor 
 : 0.57735026
2732 . professor 
 : 0.57735026
2809 . professor 
 : 0.57735026
4408 . professor 
 : 0.57735026
4636 . professor 
 : 0.57735026
5435 . professor 
 : 0.57735026
10125 . professor 
 : 0.57735026
12425 . professo

In [13]:
lda_model.save("lda_model.model")

In [ ]:
test_lines[0]

In [ ]:
len(sims[0])

In [ ]:
len(lines)

In [ ]:
sims[1][200]

In [ ]:
lda_model =  models.LdaModel.load('lda_model.model')